### Loading Cleaned Data

In [1]:
import pandas as pd

# Load cleaned data
df = pd.read_csv('../data/cleaned_events.csv')

# Convert to datetime again (safe step)
df['event_time'] = pd.to_datetime(df['event_time'])


### Defining Funnel Stages

In [2]:
# Check available event types
df['event_type'].value_counts()

event_type
view        63554512
cart         2930008
purchase      916930
Name: count, dtype: int64

In [3]:
funnel_order = ['view', 'cart', 'purchase']


### Creating Funnel Presence Flags

In [6]:
# Create a pivot table showing what events occurred in each session
session_event_flags = df.pivot_table(
    index='user_session',
    columns='event_type',
    values='event_time',
    aggfunc='count',
    fill_value=0
).astype(bool).astype(int)

session_event_flags = session_event_flags.reset_index()
session_event_flags.head()


event_type,user_session,cart,purchase,view
0,0000007c-adbf-4ed7-af17-d1fef9763d67,0,0,1
1,0000009d-1f5b-40b9-bd23-db4f3d973ae3,0,0,1
2,000001d5-f8f8-4e40-b8d6-224155713521,0,0,1
3,00000510-834a-498a-9bed-561a90c5125e,0,0,1
4,00000616-f016-4c01-b323-438486d9d3ee,0,0,1


### Defining Session Funnel Stage

In [7]:
def get_funnel_stage(row):
    if row['purchase'] == 1:
        return 'Purchased'
    elif row['cart'] == 1:
        return 'Carted'
    elif row['view'] == 1:
        return 'Viewed Only'
    else:
        return 'Other'

session_event_flags['funnel_stage'] = session_event_flags.apply(get_funnel_stage, axis=1)
session_event_flags['funnel_stage'].value_counts(normalize=True).round(2)


funnel_stage
Viewed Only    0.86
Carted         0.08
Purchased      0.06
Name: proportion, dtype: float64

### Converting Funnel Metrics

In [8]:
total_sessions = len(session_event_flags)
conversion_counts = session_event_flags['funnel_stage'].value_counts()

print("Total Sessions:", total_sessions)
print("\nFunnel Conversion Breakdown:")
print(conversion_counts)
print("\nConversion Rate:")
print(round(conversion_counts['Purchased'] / total_sessions * 100, 2), '%')


Total Sessions: 13776050

Funnel Conversion Breakdown:
funnel_stage
Viewed Only    11907857
Carted          1094979
Purchased        773214
Name: count, dtype: int64

Conversion Rate:
5.61 %


### Saving Feature Table

In [9]:
session_event_flags.to_csv('../data/session_funnel_summary.csv', index=False)
